In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
hawaii_path = "Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{hawaii_path}")
conn = engine.connect()



In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [7]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table
Measurement = Base.classes.measurement
Station= Base.classes.station

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [16]:
print(session)

In [32]:
for row in session.query(Measurement, Measurement.date).all():
    print(row)

(<sqlalchemy.ext.automap.measurement object at 0x00000194B8317520>, '2010-01-01')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B1A15E80>, '2010-01-02')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B83211F0>, '2010-01-03')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B8321040>, '2010-01-04')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B8321220>, '2010-01-06')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B8321340>, '2010-01-07')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B8321310>, '2010-01-08')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B83213D0>, '2010-01-09')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B8321E20>, '2010-01-10')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B7C34730>, '2010-01-11')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B7C346A0>, '2010-01-12')
(<sqlalchemy.ext.automap.measurement object at 0x00000194B7C34850>, '2010-01-14')
(<sqlalchemy.ext

# Exploratory Precipitation Analysis

In [26]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23',)

In [35]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
session.query(Measurement.prcp).\
 filter (Measurement.date == '2017-08-23')


# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()